# FINE-TUNING LLMs WITH HUGGING FACE

## STEP 1: installing and importing the libraries

In [1]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.5.2
Uninstalling accelerate-1.5.2:
  Successfully uninstalled accelerate-1.5.2
Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0
Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:00
  

In [2]:
!pip install huggingface_hub

In [3]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig, TrainingArguments, pipeline)

## step 2 : loading the model

In [4]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

## step 3 loading the tokenizer

In [5]:
llama_tokenizer= AutoTokenizer.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                               trust_remote_code =True )
llama_tokenizer.pad_token=llama_tokenizer.eos_token
llama_tokenizer.padding_side="right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## step 4 : Setting the trainng arguments

In [11]:
training_arguments=TrainingArguments(output_dir="./results", per_device_train_batch_size=1, max_steps=100)

## step 5 : Creating the Supervised Fine-Tuning trainer

In [12]:
llama_sft_trainer=SFTTrainer(model=llama_model,
                             args=training_arguments,
                             train_dataset=load_dataset(path="aboonaji/wiki_medical_terms_llam2_format",split="train"),
                             tokenizer=llama_tokenizer,
                             peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                             dataset_text_field="text")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## step 6 : Training the model

In [13]:
llama_sft_trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=1.7328277587890626, metrics={'train_runtime': 625.0424, 'train_samples_per_second': 0.16, 'train_steps_per_second': 0.16, 'total_flos': 2878908994584576.0, 'train_loss': 1.7328277587890626, 'epoch': 0.014575134819997084})

## step 7 : Chatting with the model

In [17]:
user_prompt=" what are the causes of rabbies"
#text generation pipeline
text_generation_pipeline=pipeline(
                                    task="text-generation",
                                    model=llama_model,
                                    tokenizer=llama_tokenizer,
                                    max_length=1000
)
model_answer=text_generation_pipeline(f"<s>[INST]{user_prompt}[/INST]")
print(model_answer[0]['generated_text'])

Device set to use cuda:0


<s>[INST] what are the causes of rabbies[/INST]  Rabies is a viral disease that affects the central nervous system of mammals, including humans and animals. everybody who comes into contact with an infected animal's saliva, urine, or other bodily fluids is at risk of contracting the disease. here are some of the most common causes of rabies:

1. Bites from infected animals: The most common way for rabies to spread is through the bite of an infected animal. Rabies is most commonly found in mammals such as dogs, bats, raccoons, skunks, and foxes.
2. Contact with infected animal's saliva: If an infected animal licks or bites someone, the virus can enter the body through the wound.
3. Infected animal's urine or feces: If an infected animal's urine or feces comes into contact with an open wound or the eyes, nose, or mouth, it can transmit the virus.
4. Organ transplant: In rare cases, rabies can be transmitted through an organ transplant from an infected donor.
5. Laboratory exposure: Rabie

In [10]:
torch.cuda.empty_cache()